In [2]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype.interfaces import freesurfer as fsr
from nipype.interfaces import brainsuite
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from shutil import copyfile
import pandas as pd
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [19]:
#Set variables
# subject_list = ['02-T2', '06-T1', '07-T2','08-T1', '09-T2', '10-T1', '11-T1', '12-T1', '13-T1', '15-T1', '16-T1', '17-T1',
#                 '18-T1', '19-T1', '20-T1', '21-T1', '22-T1', '23-T1', '24-T1', '29-T1', '30-T1', '31-T1', '32-T1',
#                 '35-T1', '36-T1', '37-T1', '38-T1', '40-T1', '102-T1', '103-T1', '104-T1', '105-T1', '106-T1', '107-T1', '108-T1', '109-T1',
#                 '110-T1', '111-T1']

#subject_list = ['08-T1']#, '20-T1','22-T1','29-T1','31-T1','35-T1']

home = '/Volumes/group/proc/TIGERanalysis'
fpath = join(home, 'qT1/subjDir')
workflow_dir = join(home, 'qT1/workflow')
data_dir = join(home, 'qT1/data')
data = '/Users/lucindasisk/Box/LS_Folders/TIGER_qT1_SOBP'

# subs = glob(data + '/*-*')
# subject_list = pd.Series(subs).str.replace(data + '/', '')
# subject_list

subs = pd.read_csv(home + '/qT1/scripts/qt1_sublist_03.31.20.txt', sep = ' ', header=None)
subject_list = subs[0].tolist()

#subject_list = ['097-T3']

In [4]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(t1=fpath+'/{subject_id}/T1_acpc.nii.gz',
               qt1=data+'/{subject_id}/fw_qT1_t1fit_final.nii.gz')

sf = Node(SelectFiles(template, 
                      base_directory = fpath),
          name = 'sf')


In [ ]:
#Skullstrip T1w image
stripT1 = Node(fsl.BET(out_file = 'T1_acpc_bet.nii.gz'),
             name = 'stripT1')

#Reorient qT1 to standard orientation
reorient = Node(fsl.Reorient2Std(out_file = 'reoriented_qT1.nii.gz',
                              output_type = 'NIFTI_GZ'),
                    name = 'reorient')

#Register qT1 to T1w
register_qt1 = Node(fsl.FLIRT(out_file = 'registered_qT1_T1acpc.nii.gz',
                              output_type = 'NIFTI_GZ',
                              out_matrix_file = 'flirt_transform_qt1.mat'),
                    name = 'register_qt1')

#Extract voxel values from qT1 beneath overlaid mask - Left Uncinate
get_voxLUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftUncinate.txt'),
               name = 'get_voxLUF')

#Extract voxel values from qT1 beneath overlaid mask - Left Cingulum Cingulate
get_voxLCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCC.txt'),
               name = 'get_voxLCC')

#Extract voxel values from qT1 beneath overlaid mask - Right Uncinate
get_voxRUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightHipp.txt'),
               name = 'get_voxRUF')

#Extract voxel values from qT1 beneath overlaid mask - Right Cingulum Cingulate
get_voxRCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftHipp.txt'),
               name = 'get_voxRCC')

In [ ]:
qt1proc_flow = Workflow(name='qt1proc_flow')
qt1proc_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
                      (sf, stripT1, [('t1', 'in_file')]),
                      (stripT1, datasink, [('out_file', '1_check_alignment')]),
                      (sf, reorient, [('qt1', 'in_file')]),
                      (reorient, datasink, [
                       ('out_file', '1_check_alignment.@par')]),
                      (reorient, register_qt1, [('out_file', 'in_file')]),
                      (stripT1, register_qt1, [('out_file', 'reference')]),
                      (register_qt1, datasink, [
                       ('out_file', '1_check_alignment.@par.@par')])
                      ])

qt1proc_flow.base_dir = workflow_dir
qt1proc_flow.write_graph(graph2use='flat')
preproc = qt1proc_flow.run('MultiProc', plugin_args={'n_procs': 4})

In [ ]:
#SelectFiles
template2 = dict(qt1r=data_dir+'/1_check_alignment/_subject_id_{subject_id}/registered_qT1.nii.gz',
               unc_maskL = fpath+'/{subject_id}/Left_Uncinate.nii.gz',
               unc_maskR = fpath+'/{subject_id}/Right_Uncinate.nii.gz',
               cc_maskL = fpath+'/{subject_id}/Left_Cingulum_Cingulate.nii.gz',
               cc_maskR = fpath+'/{subject_id}/Right_Cingulum_Cingulate.nii.gz')

# 
#             bval=fpath+'/{subject_id}/dti.bval',
#             bvec=fpath+'/{subject_id}/dti.bvec'

sf2 = Node(SelectFiles(template2, 
                      base_directory = fpath),
          name = 'sf2')

In [ ]:
get_stats=Workflow(name='get_stats')
get_stats.connect([(infosource, sf2, [('subject_id','subject_id')]),
                   (sf2, get_voxLUF, [('qt1r', 'in_file')]),
                   (sf2, get_voxLUF, [('unc_maskL', 'mask')]),
                   (get_voxLUF, datasink, [('out_file', '2_Extracted_Stats')]),
                   (sf2, get_voxLCC, [('qt1r', 'in_file')]),
                   (sf2, get_voxLCC, [('cc_maskL', 'mask')]),
                   (get_voxLCC, datasink, [('out_file', '2_Extracted_Stats.@par')]),
                   (sf2, get_voxRUF, [('qt1r', 'in_file')]),
                   (sf2, get_voxRUF, [('unc_maskR', 'mask')]),
                   (get_voxRUF, datasink, [('out_file', '2_Extracted_Stats.@par.@par')]),
                   (sf2, get_voxRCC, [('qt1r', 'in_file')]),
                   (sf2, get_voxRCC, [('cc_maskR', 'mask')]),
                   (get_voxRCC, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par')])
                  ])
get_stats.base_dir = workflow_dir
get_stats.write_graph(graph2use = 'flat')
preproc = get_stats.run('MultiProc', plugin_args={'n_procs': 4})

In [20]:
#SelectFiles
template3 = dict(t1w = home+'/TIGER_FreeSurfer/TIGER_FS_subjDir/{subject_id}/mri/brain.mgz',
                aseg = home + '/TIGER_FreeSurfer/TIGER_FS_subjDir/{subject_id}/mri/aparc.a2009s+aseg.mgz',
                qt1_reor = data_dir+'/1_check_alignment/_subject_id_{subject_id}/reoriented_qT1.nii.gz')       


sf3 = Node(SelectFiles(template3, 
                      base_directory = fpath),
          name = 'sf3')

In [6]:
#MRI Convert Nodes
convert_wb = Node(fsr.MRIConvert(in_type='mgz',
                               out_file = 'brain.nii.gz'),
               name = 'convert_wb')

convert_aseg = Node(fsr.MRIConvert(in_type='mgz',
                               out_file = 'aparc.a2009s+aseg.nii.gz'),
               name = 'convert_aseg')


In [7]:
#Extract left hippocampus - seg value 17
extract_lhipp = Node(fsl.Threshold(thresh = 16.5, 
                                   direction = 'below', 
                                   args = '-uthr 17.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_hippocampus.nii.gz'),
                        name = 'extract_lhipp')
#Extract right hippocampus - seg value 53
extract_rhipp = Node(fsl.Threshold(thresh = 52.5, 
                                   direction = 'below', 
                                   args = '-uthr 53.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_hippocampus.nii.gz'),
                    name='extract_rhipp')


In [8]:
#Extract left amygdala - seg value 18
extract_lamyg = Node(fsl.Threshold(thresh = 17.5, 
                                   direction = 'below', 
                                   args = '-uthr 18.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_amygdala.nii.gz'),
                        name = 'extract_lamyg')
#Extract right amygdala - seg value 54
extract_ramyg = Node(fsl.Threshold(thresh = 53.5, 
                                   direction = 'below', 
                                   args = '-uthr 54.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_amygdala.nii.gz'),
                    name='extract_ramyg')

In [9]:
#Extract left caudate - seg value 11
extract_lcaud = Node(fsl.Threshold(thresh = 10.5, 
                                   direction = 'below', 
                                   args = '-uthr 11.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_caudate.nii.gz'),
                        name = 'extract_lcaud')

#Extract right caudate - seg value 50
extract_rcaud = Node(fsl.Threshold(thresh = 49.5, 
                                   direction = 'below', 
                                   args = '-uthr 50.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_caudate.nii.gz'),
                    name='extract_rcaud')


In [10]:
#Extract left nucleus accumbens - seg value 26
extract_lnacc  = Node(fsl.Threshold(thresh = 25.5, 
                                   direction = 'below', 
                                   args = '-uthr 26.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_nAcc.nii.gz'),
                        name = 'extract_lnacc')

#Extract right caudate - seg value 58
extract_rnacc = Node(fsl.Threshold(thresh = 57.5, 
                                   direction = 'below', 
                                   args = '-uthr 58.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_nAcc.nii.gz'),
                    name='extract_rnacc')


In [11]:
#Extract left putamen - seg value 26
extract_lput  = Node(fsl.Threshold(thresh = 11.5, 
                                   direction = 'below', 
                                   args = '-uthr 12.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_putamen.nii.gz'),
                        name = 'extract_lput')

#Extract right caudate - seg value 51
extract_rput = Node(fsl.Threshold(thresh = 50.5, 
                                   direction = 'below', 
                                   args = '-uthr 51.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_putamen.nii.gz'),
                    name='extract_rput')

In [12]:
#Extract voxel values from qT1 beneath overlaid mask - Left Hippocampus
get_voxRhipp = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightHippocampus.txt'),
               name = 'get_voxRhipp')

#Extract voxel values from qT1 beneath overlaid mask - Right Hippocampus
get_voxLhipp = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftHippocampus.txt'),
               name = 'get_voxLhipp')

In [13]:
#Extract voxel values from qT1 beneath overlaid mask - Left Amygdala
get_voxRamyg = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightAmygdala.txt'),
               name = 'get_voxRamyg')

#Extract voxel values from qT1 beneath overlaid mask - Right Amygdala
get_voxLamyg = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftAmygdala.txt'),
               name = 'get_voxLamyg')

In [14]:
#Extract voxel values from qT1 beneath overlaid mask - Left Putamen
get_voxRput = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightPutamen.txt'),
               name = 'get_voxRput')

#Extract voxel values from qT1 beneath overlaid mask - Right Putamen
get_voxLput = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftPutamen.txt'),
               name = 'get_voxLput')

In [15]:
#Extract voxel values from qT1 beneath overlaid mask - Left NAcc
get_voxRnacc = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightNAcc.txt'),
               name = 'get_voxRnacc')

#Extract voxel values from qT1 beneath overlaid mask - Right NAcc
get_voxLnacc = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftNAcc.txt'),
               name = 'get_voxLnacc')

In [16]:
#Extract voxel values from qT1 beneath overlaid mask - Left Caudate
get_voxRcaud = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightCaudate.txt'),
               name = 'get_voxRcaud')

#Extract voxel values from qT1 beneath overlaid mask - Right Caudate
get_voxLcaud = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCaudate.txt'),
               name = 'get_voxLcaud')

In [17]:
#Reorient qT1 to standard orientation
reorient2 = Node(fsl.Reorient2Std(out_file = 'reoriented_qT1.nii.gz',
                              output_type = 'NIFTI_GZ'),
                    name = 'reorient2')
#Register qT1 to T1w
register_qt12 = Node(fsl.FLIRT(out_file = 'registered_qT1_toFreesurfer.nii.gz',
                              output_type = 'NIFTI_GZ',
                              out_matrix_file = 'flirt_transform_qt1.mat'),
                    name = 'register_qt12')

In [ ]:
hippdata_flow = Workflow(name='hippdata_flow')
hippdata_flow.connect([(infosource, sf3, [('subject_id', 'subject_id')]),
                       (sf3, convert_wb, [('t1w', 'in_file')]),
                       (sf3, convert_aseg, [('aseg', 'in_file')]),
                       (sf3, register_qt12, [('qt1_reor', 'in_file')]),
                       (convert_wb, register_qt12, [('out_file', 'reference')]),
                       (convert_aseg, extract_lhipp, [('out_file', 'in_file')]),
                       (convert_aseg, extract_rhipp, [('out_file', 'in_file')]),
                       (convert_aseg, extract_lamyg, [('out_file', 'in_file')]),
                       (convert_aseg, extract_ramyg, [('out_file', 'in_file')]),
                       (convert_aseg, extract_lput, [('out_file', 'in_file')]),
                       (convert_aseg, extract_rput, [('out_file', 'in_file')]),
                       (convert_aseg, extract_lnacc, [('out_file', 'in_file')]),
                       (convert_aseg, extract_rnacc, [('out_file', 'in_file')]),
                       (convert_aseg, extract_lcaud, [('out_file', 'in_file')]),
                       (convert_aseg, extract_rcaud, [('out_file', 'in_file')]),
                       (extract_lhipp, datasink, [('out_file', '1_check_alignment')]),
                       (extract_rhipp, datasink, [('out_file', '1_check_alignment.@par')]),
                       (extract_lamyg, datasink, [('out_file', '1_check_alignment.@par.@par')]),
                       (extract_ramyg, datasink, [('out_file', '1_check_alignment.@par.@par.@par')]),
                       (extract_lput, datasink, [('out_file', '1_check_alignment.@par.@par.@par.@par')]),
                       (extract_rput, datasink, [('out_file', '1_check_alignment.@par.@par.@par.@par.@par')]),
                       (extract_lnacc, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par')]),
                       (extract_rnacc, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par')]),
                       (extract_lcaud, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (extract_rcaud, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (register_qt12, datasink,[('out_file', '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (register_qt12, get_voxRhipp, [('out_file', 'in_file')]),
                       (extract_rhipp, get_voxRhipp, [('out_file', 'mask')]),
                       (register_qt12, get_voxLhipp, [('out_file', 'in_file')]),
                       (extract_lhipp, get_voxLhipp, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRamyg, [('out_file', 'in_file')]),
                       (extract_ramyg, get_voxRamyg, [('out_file', 'mask')]),
                       (register_qt12, get_voxLamyg, [('out_file', 'in_file')]),
                       (extract_lamyg, get_voxLamyg, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRput, [('out_file', 'in_file')]),
                       (extract_rput, get_voxRput, [('out_file', 'mask')]),
                       (register_qt12, get_voxLput, [('out_file', 'in_file')]),
                       (extract_lput, get_voxLput, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRcaud, [('out_file', 'in_file')]),
                       (extract_rcaud, get_voxRcaud, [('out_file', 'mask')]),
                       (register_qt12, get_voxLcaud, [('out_file', 'in_file')]),
                       (extract_lcaud, get_voxLcaud, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRnacc, [('out_file', 'in_file')]),
                       (extract_rnacc, get_voxRnacc, [('out_file', 'mask')]),
                       (register_qt12, get_voxLnacc, [('out_file', 'in_file')]),
                       (extract_lnacc, get_voxLnacc, [('out_file', 'mask')]),
                       
                       (get_voxLhipp, datasink, [('out_file', '2_Extracted_Stats')]),
                       (get_voxRhipp, datasink, [('out_file', '2_Extracted_Stats.@par')]),
                       (get_voxLamyg, datasink, [('out_file', '2_Extracted_Stats.@par.@par')]),
                       (get_voxRamyg, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par')]),
                       (get_voxLput, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par')]),
                       (get_voxRput, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par')]),
                       (get_voxLnacc, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par')]),
                       (get_voxRnacc, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par')]),
                       (get_voxLcaud, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (get_voxRcaud, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par.@par.@par')])
                      ])

hippdata_flow.base_dir = workflow_dir
hippdata_flow.write_graph(graph2use='flat')
preproc = hippdata_flow.run('MultiProc', plugin_args={'n_procs': 4})

200331-14:24:59,572 nipype.workflow DEBUG:
	 (hippdata_flow.infosource, hippdata_flow.sf3): No edge data
200331-14:24:59,573 nipype.workflow DEBUG:
	 (hippdata_flow.infosource, hippdata_flow.sf3): new edge data: {'connect': [('subject_id', 'subject_id')]}
200331-14:24:59,574 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.convert_wb): No edge data
200331-14:24:59,575 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.convert_wb): new edge data: {'connect': [('t1w', 'in_file')]}
200331-14:24:59,576 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.convert_aseg): No edge data
200331-14:24:59,577 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.convert_aseg): new edge data: {'connect': [('aseg', 'in_file')]}
200331-14:24:59,579 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.register_qt12): No edge data
200331-14:24:59,579 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.register_qt12): new edge data: {'connect': [('qt1_reor', 'in

200331-14:24:59,626 nipype.workflow DEBUG:
	 (hippdata_flow.extract_lhipp, hippdata_flow.get_voxLhipp): new edge data: {'connect': [('out_file', 'mask')]}
200331-14:24:59,627 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxRamyg): No edge data
200331-14:24:59,627 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxRamyg): new edge data: {'connect': [('out_file', 'in_file')]}
200331-14:24:59,628 nipype.workflow DEBUG:
	 (hippdata_flow.extract_ramyg, hippdata_flow.get_voxRamyg): No edge data
200331-14:24:59,629 nipype.workflow DEBUG:
	 (hippdata_flow.extract_ramyg, hippdata_flow.get_voxRamyg): new edge data: {'connect': [('out_file', 'mask')]}
200331-14:24:59,630 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxLamyg): No edge data
200331-14:24:59,630 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxLamyg): new edge data: {'connect': [('out_file', 'in_file')]}
200331-14:24:59,631 nip

200331-14:24:59,686 nipype.workflow DEBUG:
	 processing node: hippdata_flow.extract_rput
200331-14:24:59,687 nipype.workflow DEBUG:
	 processing node: hippdata_flow.extract_lput
200331-14:24:59,687 nipype.workflow DEBUG:
	 processing node: hippdata_flow.extract_ramyg
200331-14:24:59,688 nipype.workflow DEBUG:
	 processing node: hippdata_flow.extract_lamyg
200331-14:24:59,688 nipype.workflow DEBUG:
	 processing node: hippdata_flow.extract_rhipp
200331-14:24:59,689 nipype.workflow DEBUG:
	 processing node: hippdata_flow.extract_lhipp
200331-14:24:59,689 nipype.workflow DEBUG:
	 processing node: hippdata_flow.convert_wb
200331-14:24:59,690 nipype.workflow DEBUG:
	 processing node: hippdata_flow.register_qt12
200331-14:24:59,690 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxLnacc
200331-14:24:59,691 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxRnacc
200331-14:24:59,692 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxLcaud
200331-14:24:59,69

200331-14:25:35,835 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 106-T1
200331-14:25:35,836 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_106-T1
200331-14:25:35,843 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 107-T1
200331-14:25:35,844 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_107-T1
200331-14:25:35,853 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 108-T1
200331-14:25:35,854 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_108-T1
200331-14:25:35,862 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 109-T1
200331-14:25:35,863 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_109-T1
200331-14:25:35,871 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 109-T3
200331-14:25:35,872 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_109-T3
200331-14:25:35,880 nipype.workflow DEBUG:
	 [Node

200331-14:25:36,262 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_215-T3
200331-14:25:36,270 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 22-T1
200331-14:25:36,271 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_22-T1
200331-14:25:36,280 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 23-T1
200331-14:25:36,281 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_23-T1
200331-14:25:36,290 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 24-T1
200331-14:25:36,291 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_24-T1
200331-14:25:36,298 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 28-T2
200331-14:25:36,300 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_28-T2
200331-14:25:36,310 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 29-T1
200331-14:25:36,311 nipype.workflow DEBUG:
	 Parameterizati

...

200331-14:28:09,515 nipype.workflow DEBUG:
	 Cannot allocate job 1175 (0.20GB, 1 threads).
200331-14:28:09,516 nipype.workflow DEBUG:
	 Cannot allocate job 1200 (0.20GB, 1 threads).
200331-14:28:09,517 nipype.workflow DEBUG:
	 Cannot allocate job 1225 (0.20GB, 1 threads).
200331-14:28:09,517 nipype.workflow DEBUG:
	 Cannot allocate job 1250 (0.20GB, 1 threads).
200331-14:28:09,518 nipype.workflow DEBUG:
	 Cannot allocate job 1275 (0.20GB, 1 threads).
200331-14:28:09,519 nipype.workflow DEBUG:
	 Cannot allocate job 1300 (0.20GB, 1 threads).
200331-14:28:09,519 nipype.workflow DEBUG:
	 Cannot allocate job 1325 (0.20GB, 1 threads).
200331-14:28:09,520 nipype.workflow DEBUG:
	 Cannot allocate job 1350 (0.20GB, 1 threads).
200331-14:28:09,521 nipype.workflow DEBUG:
	 Cannot allocate job 1375 (0.20GB, 1 threads).
200331-14:28:09,521 nipype.workflow DEBUG:
	 Cannot allocate job 1400 (0.20GB, 1 threads).
200331-14:28:09,522 nipype.workflow DEBUG:
	 Cannot allocate job 1425 (0.20GB, 1 threads).

200331-14:28:11,513 nipype.utils DEBUG:
	 Loading pkl: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_55-T1/convert_aseg/result_convert_aseg.pklz
200331-14:28:11,885 nipype.workflow DEBUG:
	 output: out_file
200331-14:28:11,886 nipype.workflow DEBUG:
	 [Node] extract_rhipp - setting input in_file = /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_55-T1/convert_aseg/aparc.a2009s+aseg.nii.gz
200331-14:28:13,101 nipype.workflow DEBUG:
	 [Node] Hashes: [('args', '-uthr 53.5'), ('direction', 'below'), ('in_file', ('/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_55-T1/convert_aseg/aparc.a2009s+aseg.nii.gz', '8286ed2674047546c3c47b3b2238c32b')), ('out_file', 'right_hippocampus.nii.gz'), ('output_type', 'NIFTI_GZ'), ('thresh', '52.5000000000')], 3f4339b90c112be3db44629a7fbc1f1c, /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_55-T1/extract_rhipp/_0x3f4339b90c112be3db44629a7fbc1f1c.json, ['/Volu

200331-14:28:13,160 nipype.workflow DEBUG:
	 Cannot allocate job 1800 (0.20GB, 1 threads).
200331-14:28:13,161 nipype.workflow DEBUG:
	 Cannot allocate job 1825 (0.20GB, 1 threads).
200331-14:28:13,161 nipype.workflow DEBUG:
	 Cannot allocate job 1850 (0.20GB, 1 threads).
200331-14:28:13,162 nipype.workflow DEBUG:
	 Cannot allocate job 1875 (0.20GB, 1 threads).
200331-14:28:13,162 nipype.workflow DEBUG:
	 Cannot allocate job 1900 (0.20GB, 1 threads).
200331-14:28:13,163 nipype.workflow DEBUG:
	 Cannot allocate job 1925 (0.20GB, 1 threads).
200331-14:28:13,164 nipype.workflow DEBUG:
	 Cannot allocate job 1950 (0.20GB, 1 threads).
200331-14:28:13,165 nipype.workflow DEBUG:
	 Cannot allocate job 1975 (0.20GB, 1 threads).
200331-14:28:13,165 nipype.workflow DEBUG:
	 Cannot allocate job 2000 (0.20GB, 1 threads).
200331-14:28:13,166 nipype.workflow DEBUG:
	 Cannot allocate job 2025 (0.20GB, 1 threads).
200331-14:28:13,167 nipype.workflow DEBUG:
	 Cannot allocate job 2050 (0.20GB, 1 threads).

200331-14:28:17,21 nipype.workflow DEBUG:
	 Cannot allocate job 200 (0.20GB, 1 threads).
200331-14:28:17,22 nipype.workflow DEBUG:
	 Cannot allocate job 225 (0.20GB, 1 threads).
200331-14:28:17,23 nipype.workflow DEBUG:
	 Cannot allocate job 250 (0.20GB, 1 threads).
200331-14:28:17,23 nipype.workflow DEBUG:
	 Cannot allocate job 275 (0.20GB, 1 threads).
200331-14:28:17,24 nipype.workflow DEBUG:
	 Cannot allocate job 300 (0.20GB, 1 threads).
200331-14:28:17,25 nipype.workflow DEBUG:
	 Cannot allocate job 325 (0.20GB, 1 threads).
200331-14:28:17,25 nipype.workflow DEBUG:
	 Cannot allocate job 350 (0.20GB, 1 threads).
200331-14:28:17,26 nipype.workflow DEBUG:
	 Cannot allocate job 375 (0.20GB, 1 threads).
200331-14:28:17,27 nipype.workflow DEBUG:
	 Cannot allocate job 400 (0.20GB, 1 threads).
200331-14:28:17,27 nipype.workflow DEBUG:
	 Cannot allocate job 425 (0.20GB, 1 threads).
200331-14:28:17,28 nipype.workflow DEBUG:
	 Cannot allocate job 450 (0.20GB, 1 threads).
200331-14:28:17,28 ni

	 key: 1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par files: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_56-T1/register_qt12/registered_qT1_toFreesurfer.nii.gz
200331-14:28:18,129 nipype.workflow DEBUG:
	 input: reference
200331-14:28:18,130 nipype.workflow DEBUG:
	 results file: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_55-T1/convert_wb/result_convert_wb.pklz
200331-14:28:18,131 nipype.utils DEBUG:
	 Loading pkl: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_55-T1/convert_wb/result_convert_wb.pklz
200331-14:28:18,229 nipype.interface DEBUG:
	 copyfile: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_56-T1/register_qt12/registered_qT1_toFreesurfer.nii.gz /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_56-T1/registered_qT1_toFreesurfer.nii.gz
200331-14:28:18,236 nipype.utils DEBUG:
	 /Volumes/group/proc/TIGERanalysis/qT1/data/1_che